In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

In [3]:
import os

data_path = "/content/drive/MyDrive/DS340_Final_Project/Audio_Song_Actors_01-24"
base_out = "/content/drive/MyDrive/DS340_Final_Project/augmented_spectrograms_noise"

os.makedirs(base_out, exist_ok=True)

In [4]:
noise_levels = [0.005, 0.01, 0.02, 0.05,0.1]

In [5]:
emotion_map = {
    "01":"neutral","02":"calm","03":"happy","04":"sad",
    "05":"angry","06":"fearful","07":"disgust","08":"surprised"
}

In [ ]:
for eps in noise_levels:
    # 1) create a noise‑level folder
    noise_dir = os.path.join(data_path, f"noise_{eps}")
    os.makedirs(noise_dir, exist_ok=True)

    # 2) inside it, make one subfolder per emotion
    for emo in emotion_map.values():
        os.makedirs(os.path.join(noise_dir, emo), exist_ok=True)

    # 3) process each actor / file
    for actor in tqdm(os.listdir(data_path), desc=f"Noise={eps}"):
        actor_path = os.path.join(data_path, actor)
        if not os.path.isdir(actor_path):
            continue

        for fn in os.listdir(actor_path):
            if not fn.endswith(".wav"):
                continue

            # pull out emotion code from filename
            emotion_code = fn.split("-")[2]
            emotion = emotion_map.get(emotion_code)
            if emotion is None:
                continue

            # load audio
            wav_path = os.path.join(actor_path, fn)
            y, sr = librosa.load(wav_path, sr=None)

            # add white noise
            noise = np.random.randn(len(y))
            y_noisy = y + eps * noise

            # compute mel‑spectrogram
            S = librosa.feature.melspectrogram(
                y=y_noisy, sr=sr, n_mels=128, fmax=8000
            )
            S_db = librosa.power_to_db(S, ref=np.max)

            # plot & save
            plt.figure(figsize=(3,3))
            librosa.display.specshow(S_db, sr=sr,
                                     x_axis='time', y_axis='mel')
            plt.axis('off')

            outname = fn.replace(".wav", f"_noise{eps}.png")
            save_path = os.path.join(noise_dir, emotion, outname)
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
            plt.close()

Noise=0.02:  85%|████████▌ | 23/27 [04:09<00:45, 11.36s/it]